In [19]:
#import dependencies

import pandas as pd
from bs4 import BeautifulSoup as bs
#from bs4.element import Comment
#from splinter import Browser
import requests
#import time
#import numpy as np


In [20]:
#import sqlalchemy
from sqlalchemy import create_engine


In [21]:
engine = create_engine('postgresql://<username>:<password>@localhost:5432/firedata_db')

In [22]:
#import and establish base for which classes will be constructed
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [5]:
#import and establish base for which classes will be constructed
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [18]:
#create classes

class Fire(Base):
    __tablename__ = 'clean_data'
    id = Column(Integer, primary_key= True)
    title = Column(String(255))
    published = Column(String(255))
    lat = Column(String(255))
    lon = Column(String(255))
    link = Column(String(255))
    description = Column(String(255))
    cause = Column(String(255))
    size = Column(Integer)

C:\Users\sharr\anaconda3\lib\site-packages\sqlalchemy\ext\declarative\clsregistry.py:125: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.Fire, and will be replaced in the string-lookup table.
  util.warn(


In [12]:
 #Create Database Connection
# ----------------------------------
# Creates a connection to our DB
#engine = create_engine("sqlite:///fire.sqlite")
conn = engine.connect()

In [13]:
# Create a "Metadata" Layer That Abstracts our SQL Database
# ----------------------------------
# Create (if not already in existence) the tables associated with our classes.
Base.metadata.create_all(engine)

# Use this to clear out the db
# ----------------------------------
# Base.metadata.drop_all(engine) 

In [14]:
# Create a Session Object to Connect to DB
# ----------------------------------
# Session is a temporary binding to our DB
from sqlalchemy.orm import Session
session = Session(bind=engine)

In [15]:
# Perform a simple query of the database
fire_list = session.query(clean_data)
for fire in fire_list:
    print(clean_data.title)

NameError: name 'clean_data' is not defined

In [16]:
pd.read_sql_query('select * from clean_data', con=engine).head()

NameError: name 'pd' is not defined

## Extract Data

In [3]:
#datasource url 
url = "https://inciweb.nwcg.gov/feeds/rss/incidents/"

In [4]:
# instanciate landing dataframe
starting_data = pd.DataFrame()

# request xml code from url
xml_data = requests.get(url).content

#parse the xml response
soup = bs(xml_data, "xml")
     
# Find all text in the data
texts = str(soup.findAll(text=True)).replace('\\n','')
    
#Find the tag/child
child = soup.find("item")

# instatiate column lists
title = []
published = []
lat = []
lon = []
link = []
description = []

#loop trough each "item" in the xml response and store the target data
while True:    
    try:
        title.append(" ".join(child.find('title')))
    except:
         title.append(" ")
            
    try:
        published.append(" ".join(child.find('published')))
    except:
        published.append(" ")
        
    try:
        lat.append(" ".join(child.find('geo:lat')))
    except:
        lat.append(" ")
            
    try:
        lon.append(" ".join(child.find('geo:long')))
    except:
        lon.append(" ")
        
    try:
        link.append(" ".join(child.find('link')))
    except:
        link.append(" ")
        
    try:
        description.append(" ".join(child.find('description')))
    except:
        description.append(" ")
        
    try:   
        # Next sibling of child, here: 'item' 
        child = child.find_next_sibling('item')
    except:
        break
    
    #create dataframe
    data = pd.DataFrame({"title":title,
                                    "published":published,
                                    "lat":lat,
                                    "lon":lon,
                                    "link":link,
                                    "description": description})
    starting_data = starting_data.append(data, ignore_index = True)

In [5]:
# drop duplicate rows
unique_data = starting_data.drop_duplicates(keep="first",ignore_index="True")
unique_data.head(15)

title  \
0      Fort Wolters Prescribed Fire (Prescribed Fire)   
1                              SQF Complex (Wildfire)   
2                        Cameron Peak Fire (Wildfire)   
3                               Creek Fire (Wildfire)   
4   Fall Prescribed Fire Projects 2020 (Prescribed...   
5                       Slater/Devil Fires (Wildfire)   
6                            North Complex (Wildfire)   
7                       Mountain View Fire (Wildfire)   
8                             Laura 2 Fire (Wildfire)   
9                              Grouse Fire (Wildfire)   
10                        Middle Fork Fire (Wildfire)   
11                        Rattlesnake Fire (Wildfire)   
12  Shasta-Trinity RX Burning 2020/2021 (Prescribe...   
13     Woodhead BAER (Burned Area Emergency Response)   
14                            Moraine Fire (Wildfire)   

                           published              lat               lon  \
0   Tue, 01 Dec 2020 09:14:28 -06:00  32.869166666667           -98.045   
1   Tue, 01 Dec 2020 07:35:02 -06:00           36.255  -118.49666666667   
2   Mon, 30 Nov 2020 21:02:55 -06:00  40.608611111111  -105.87916666667   
3   Mon, 30 Nov 2020 18:18:29 -06:00  37.201111111111  -119.27166666667   
4   Mon, 30 Nov 2020 14:20:28 -06:00  44.417777777778  -118.95138888889   
5   Sat, 28 Nov 2020 14:40:10 -06:00  41.766111111111          -123.375   
6   Wed, 25 Nov 2020 11:56:38 -06:00  40.090833333333  -120.93111111111   
7   Wed, 25 Nov 2020 10:34:07 -06:00           38.515  -119.46527777778   
8   Tue, 24 Nov 2020 16:31:07 -06:00  40.334166666667          -120.115   
9   Tue, 24 Nov 2020 12:49:19 -06:00  43.540555555556  -115.17277777778   
10  Mon, 23 Nov 2020 17:35:42 -06:00  40.627777777778  -106.79444444444   
11  Mon, 23 Nov 2020 13:29:05 -06:00  36.419722222222  -118.44833333333   
12  Mon, 23 Nov 2020 11:43:12 -06:00  41.371944444444  -121.97138888889   
13  Mon, 23 Nov 2020 10:43:36 -06:00  44.756111111111  -116.87666666667   
14  Sun, 22 Nov 2020 18:09:42 -06:00  36.717777777778  -118.55722222222   

                                      link  \
0   http://inciweb.nwcg.gov/incident/7272/   
1   http://inciweb.nwcg.gov/incident/7048/   
2   http://inciweb.nwcg.gov/incident/6964/   
3   http://inciweb.nwcg.gov/incident/7147/   
4   http://inciweb.nwcg.gov/incident/7216/   
5   http://inciweb.nwcg.gov/incident/7173/   
6   http://inciweb.nwcg.gov/incident/6997/   
7   http://inciweb.nwcg.gov/incident/7270/   
8   http://inciweb.nwcg.gov/incident/7269/   
9   http://inciweb.nwcg.gov/incident/7154/   
10  http://inciweb.nwcg.gov/incident/7153/   
11  http://inciweb.nwcg.gov/incident/7131/   
12  http://inciweb.nwcg.gov/incident/7268/   
13  http://inciweb.nwcg.gov/incident/7262/   
14  http://inciweb.nwcg.gov/incident/7089/   

                                          description  
0   Texas A&M Forest Service is assisting military...  
1   Operations Map |  Fire History Map |   Land Ow...  
2   The Southern Area Gold Type 2 Incident Managem...  
3                                                      
4    Please take a moment to look at the links sec...  
5       Please visit our incident's Facebook page ...  
6    The incident command post (ICP) for North Com...  
7   The Mountain View fire is 70% contained at 20,...  
8   The fire is fully contained, and the Fort Sage...  
9    The Grouse Fire started Sept. 6, 2020, about ...  
10  Middle Fork Fact Sheet - 11/02/2020, 2:30 p.m....  
11  The Rattlesnake Fire was discovered on August ...  
12   See the 'Announcements' and 'News' Tabs for t...  
13    The Woodhead Fire burned almost 100,000 acre...  
14       The Moraine Fire was discovered on August...

In [6]:
# go the the link url for each rown and extract additional data (cause, size)

#instatiate landing lists
causes = []
sizes = []

#loop through each row of data
for x in range(len(unique_data)):
    #find the link in the row
    url = starting_data.loc[x,"link"]
    #go to the page and grap all the tables
    tables = pd.read_html(url)
    
    #the number of tables the page has will determine which tables we look in for data.
    #if there are more than one tables...
    if len(tables)>1:
        try:
            # find the "cause" in the first table on the page (if it exists)
            cause = tables[0].iloc[2,1]
        except:
            cause = "unknown"
    
        try:
            # find the "size" in the second table on the page (if it exists)
            size = tables[1].loc[(tables[1][0]) == "Size",1].item()            
        except:
            size = "unknown"
    #if there is only one table on the page...
    else:
        try:
            # find the "cause" in the first table on the page (if it exists)
            cause = tables[0].iloc[1,1]
            # no size data is available
            size = "n/a"
        except:
            cause = "unknown"
    
    #add cause and size to their lists
    causes.append(cause)
    sizes.append(size)
    
    #print progress
    print(f"{x+1} of {len(unique_data)}")


1 of 136
2 of 136
3 of 136
4 of 136
5 of 136
6 of 136
7 of 136
8 of 136
9 of 136
10 of 136
11 of 136
12 of 136
13 of 136
14 of 136
15 of 136
16 of 136
17 of 136
18 of 136
19 of 136
20 of 136
21 of 136
22 of 136
23 of 136
24 of 136
25 of 136
26 of 136
27 of 136
28 of 136
29 of 136
30 of 136
31 of 136
32 of 136
33 of 136
34 of 136
35 of 136
36 of 136
37 of 136
38 of 136
39 of 136
40 of 136
41 of 136
42 of 136
43 of 136
44 of 136
45 of 136
46 of 136
47 of 136
48 of 136
49 of 136
50 of 136
51 of 136
52 of 136
53 of 136
54 of 136
55 of 136
56 of 136
57 of 136
58 of 136
59 of 136
60 of 136
61 of 136
62 of 136
63 of 136
64 of 136
65 of 136
66 of 136
67 of 136
68 of 136
69 of 136
70 of 136
71 of 136
72 of 136
73 of 136
74 of 136
75 of 136
76 of 136
77 of 136
78 of 136
79 of 136
80 of 136
81 of 136
82 of 136
83 of 136
84 of 136
85 of 136
86 of 136
87 of 136
88 of 136
89 of 136
90 of 136
91 of 136
92 of 136
93 of 136
94 of 136
95 of 136
96 of 136
97 of 136
98 of 136
99 of 136
100 of 136
101 of 1

## Clean/Transform Data 

In [7]:
# if cause has the word "investigation" in it, set cause to "unknown"
for y in range(len(causes)):
    if "Investigation" in causes[y]:
        causes[y] = "Unknown"

# remove the word "Acres" from the size data
sizes = [s.replace(" Acres","") for s in sizes]

In [8]:
# add causes and sizes to the dataframe
unique_data["cause"] = causes
unique_data["size"] = sizes

<ipython-input-8-8ab9e14f991d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_data["cause"] = causes
<ipython-input-8-8ab9e14f991d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_data["size"] = sizes


In [9]:
# see the counts of each fire cause for reference
grouped_df = unique_data.groupby(["cause"])
grouped_df.count()

title  published  lat  lon  link  \
cause                                                                         
Burned Area Emergency Response                 3          3    3    3     3   
Human                                          7          7    7    7     7   
Lightning                                     38         38   38   38    38   
Prescribed Fire                               16         16   16   16    16   
Prescribed Fire To Reduce Wildfire Hazard     16         16   16   16    16   
Unknown                                       56         56   56   56    56   

                                           description  size  
cause                                                         
Burned Area Emergency Response                       3     3  
Human                                                7     7  
Lightning                                           38    38  
Prescribed Fire                                     16    16  
Prescribed Fire To Reduce Wildfire Hazard           16    16  
Unknown                                             56    56

In [10]:
# save the dataframe as "clean_data"

clean_data = unique_data
clean_data.head()

title  \
0     Fort Wolters Prescribed Fire (Prescribed Fire)   
1                             SQF Complex (Wildfire)   
2                       Cameron Peak Fire (Wildfire)   
3                              Creek Fire (Wildfire)   
4  Fall Prescribed Fire Projects 2020 (Prescribed...   

                          published              lat               lon  \
0  Tue, 01 Dec 2020 09:14:28 -06:00  32.869166666667           -98.045   
1  Tue, 01 Dec 2020 07:35:02 -06:00           36.255  -118.49666666667   
2  Mon, 30 Nov 2020 21:02:55 -06:00  40.608611111111  -105.87916666667   
3  Mon, 30 Nov 2020 18:18:29 -06:00  37.201111111111  -119.27166666667   
4  Mon, 30 Nov 2020 14:20:28 -06:00  44.417777777778  -118.95138888889   

                                     link  \
0  http://inciweb.nwcg.gov/incident/7272/   
1  http://inciweb.nwcg.gov/incident/7048/   
2  http://inciweb.nwcg.gov/incident/6964/   
3  http://inciweb.nwcg.gov/incident/7147/   
4  http://inciweb.nwcg.gov/incident/7216/   

                                         description  \
0  Texas A&M Forest Service is assisting military...   
1  Operations Map |  Fire History Map |   Land Ow...   
2  The Southern Area Gold Type 2 Incident Managem...   
3                                                      
4   Please take a moment to look at the links sec...   

                                       cause     size  
0  Prescribed Fire To Reduce Wildfire Hazard      449  
1  Prescribed Fire To Reduce Wildfire Hazard      449  
2                                  Lightning  174,178  
3  Prescribed Fire To Reduce Wildfire Hazard      449  
4                                  Lightning  174,178